In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "offensive")

README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/93.7k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/122k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/860 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1324 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11916
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1324
    })
})

In [5]:
id2label = {0: "NotOffensive", 1: "Offensive"}
label2id = {"NotOffensive":0, "offensive":1}

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [8]:
model_checkpoint = 'distilbert-base-uncased'

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels = 2,
    id2label = id2label,
    label2id = label2id
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def tokenize_function(data):
  return tokenizer(data["text"], padding = "max_length", truncation = True)

Tokenized_data = dataset.map(tokenize_function,batched = True)

Map:   0%|          | 0/11916 [00:00<?, ? examples/s]

Map:   0%|          | 0/860 [00:00<?, ? examples/s]

Map:   0%|          | 0/1324 [00:00<?, ? examples/s]

In [12]:
import evaluate

accuracy = evaluate.load("accuracy")

In [13]:
import numpy as np

In [35]:
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=p.label_ids)}


In [15]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [16]:
peft = LoraConfig(
    task_type= "SEQ_CLS",
    r=4,
    lora_alpha = 32,
    lora_dropout = 0.01,
    target_modules = ['q_lin']
)

In [17]:
peft

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [18]:
model = get_peft_model(model, peft)

In [19]:
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [20]:
from transformers import TrainingArguments, Trainer

In [31]:
training_args = TrainingArguments(
    output_dir = "/content/logs",
    learning_rate = 1e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size = 8,
    num_train_epochs = 5,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Tokenized_data['train'],
    eval_dataset = Tokenized_data['validation'],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

<ipython-input-36-68f9f1665aac>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.610500,0.549541,{'accuracy': 0.7515105740181269}
2,0.584300,0.557035,{'accuracy': 0.7673716012084593}
3,0.561500,0.568429,{'accuracy': 0.7749244712990937}
4,0.535100,0.572076,{'accuracy': 0.7817220543806647}
5,0.482000,0.608261,{'accuracy': 0.7787009063444109}


TrainOutput(global_step=14895, training_loss=0.5602428373533354, metrics={'train_runtime': 2251.5057, 'train_samples_per_second': 26.462, 'train_steps_per_second': 6.616, 'total_flos': 8007532232785920.0, 'train_loss': 0.5602428373533354, 'epoch': 5.0})

In [40]:
import torch

In [43]:
test_tweets = [
    "You're such a loser, no one likes you.",
    "I really enjoyed the concert last night!",
    "Shut up already. No one cares what you think.",
    "Happy birthday! Wishing you all the best 🎉",
    "This is the dumbest thing I've ever seen.",
    "Grateful for everyone who supported me!",
    "You look disgusting. Stay home.",
    "The sunset today was absolutely beautiful!",
    "Go cry somewhere else, weakling.",
    "Just finished a great workout. Feeling amazing!"
]


for text in test_tweets:

    inputs = tokenizer.encode(text, return_tensors="pt").to(model.device)
    logits = model(inputs).logits

    predictions = torch.argmax(logits, dim=1)

    print(text + " - " + id2label[predictions.item()])


----------------------------
You're such a loser, no one likes you. - Offensive
I really enjoyed the concert last night! - NotOffensive
Shut up already. No one cares what you think. - NotOffensive
Happy birthday! Wishing you all the best 🎉 - NotOffensive
This is the dumbest thing I've ever seen. - Offensive
Grateful for everyone who supported me! - NotOffensive
You look disgusting. Stay home. - Offensive
The sunset today was absolutely beautiful! - NotOffensive
Go cry somewhere else, weakling. - NotOffensive
Just finished a great workout. Feeling amazing! - NotOffensive


In [45]:
test_dataset = Tokenized_data["test"]
test_results = trainer.evaluate(test_dataset)
print(test_results)


pred = trainer.predict(test_dataset)
print(pred.metrics)


{'eval_loss': 0.4943658113479614, 'eval_accuracy': {'accuracy': 0.7790697674418605}, 'eval_runtime': 13.2388, 'eval_samples_per_second': 64.961, 'eval_steps_per_second': 8.158, 'epoch': 5.0}
{'test_loss': 0.4943658113479614, 'test_accuracy': {'accuracy': 0.7790697674418605}, 'test_runtime': 13.108, 'test_samples_per_second': 65.609, 'test_steps_per_second': 8.239}
